In [3]:
#meta 10/11/2020 myTry Ch 4&5. Image Classification (multilabel) - MNIST full.

#change log
#prev 10/11/2020 TRAIN MNIST CLASSIFIER, WITH AND WITHOUT PRETRAINED MODEL

#here 10/12/2020 PICKLE DATA FOR REUSE
#      Save dls_train and dls_test 
#      Load pre-pickled data if available.


In [4]:
#load modules
from fastbook import *
from fastai.vision.all import *

# Image Classification - MNIST
full dataset

## 0. Load Data

If no pre-pickled data, then get it from source and pickle for future reuse.

In [5]:
#!pwd => /home/jupyter
try:
    dls_train = torch.load('myData/mnist_full_dls_train.pkl') #class fastai.data.core.DataLoaders
    dls_test = torch.load('myData/mnist_full_dls_test.pkl') 
except:
    flg_data_pkl = False

In [6]:
if not flg_data_pkl:
    print(URLs.MNIST)
    
    path = untar_data(URLs.MNIST)
    #set path reference
    Path.BASE_PATH = path

    #see what's in our dataset -> 2 directories
    print("Path folders: ", path.ls())

https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz
Path folders:  (#2) [Path('testing'),Path('training')]


Files organization: train and test folders, then each digit in its own folder.

In [7]:
if not flg_data_pkl:
    #files structure
    print((path/"training").ls()) #class fastcore.foundation.L
    print((path/"training/7").ls())
    

(#10) [Path('training/3'),Path('training/8'),Path('training/5'),Path('training/4'),Path('training/9'),Path('training/7'),Path('training/2'),Path('training/6'),Path('training/1'),Path('training/0')]
(#6265) [Path('training/7/11328.png'),Path('training/7/30538.png'),Path('training/7/36916.png'),Path('training/7/6986.png'),Path('training/7/981.png'),Path('training/7/50464.png'),Path('training/7/19370.png'),Path('training/7/18720.png'),Path('training/7/23309.png'),Path('training/7/9905.png')...]


In [20]:
if not flg_data_pkl:
     #look at one image
    fname = (path/"training/7").ls()[0]
    
Image.open(fname)

## 1. Preprocess Data
#### Train set
Get all files from `training` folder and make them available as train and valid.

In [8]:
if not flg_data_pkl:
    digits_train = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(valid_pct = 0.2, seed=42),
                 get_y = parent_label, 
                 item_tfms=Resize(128))#,
                 #batch_tfms=aug_transforms(size=224, min_scale=0.75)
        
    dls_train = digits_train.dataloaders(path/"training")
    
    print(digits_train.__class__, dls_train.__class__)

<class 'fastai.data.block.DataBlock'> <class 'fastai.data.core.DataLoaders'>


#### Test set
Get all files from `testing` folder.  Don't use it for training.  Hold it for measuring model performance.

In [9]:
if not flg_data_pkl:
    digits_test = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 get_y = parent_label, 
                 item_tfms=Resize(128))
    
    dls_test = digits_test.dataloaders(path/"testing")

Pickle files for reuse and saving GCP money

In [13]:
if not flg_data_pkl:
    #torch.save( digits_train, 'myData/mnist_full_datablock_train.pkl') #class fastai.data.block.DataBlock
    #torch.save( digits_test, 'myData/mnist_full_datablock_test.pkl') 
    torch.save( dls_train, 'myData/mnist_full_dls_train.pkl') #class fastai.data.core.DataLoaders
    torch.save( dls_test, 'myData/mnist_full_dls_test.pkl')


In [ ]:
df100

#### Checking and Debugging a DataBlock

In [ ]:
dls_train.show_batch(nrows=2, ncols=3)

## 2. Train model
### 2.1 Using fast.ai cnn learner, without pretrained model

In [ ]:
#pg.180
my_learn = cnn_learner(dls_train, resnet18, pretrained=False, metrics=error_rate)
my_learn.fit_one_cycle(1, 0.1)

#### 2.1a Check Model Performance
on the validation set

In [ ]:
x_train,y_train = dls_train.one_batch()

In [ ]:
#preview actuals
y_train

In [ ]:
#preview first prediction
preds,_ = my_learn.get_preds(dl=[(x_train,y_train)])
preds[0]

In [ ]:
len(preds[0]),preds[0].sum()

In [ ]:
print("Loss function:")
my_learn.loss_func

In [ ]:
#learn.metrics[0].__class__ #fastai.learner.AvgMetric
print("Metric: error rate")
my_learn.metrics[0].value

## Model Interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(my_learn)
interp.plot_top_losses(5, nrows=1)

In [ ]:
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp.most_confused(min_val=6)

## 3. Improving Our Model
#### The Learning Rate Finder

In [ ]:
dls = dls_train.clone()

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
lr_min,lr_steep = learn.lr_find()

In [ ]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fine_tune(2, base_lr=5e-3)

In [ ]:
$ac redo above with diff rate

#### Unfreezing and Transfer Learning

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fit_one_cycle(3, 5e-3)

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()

In [ ]:
#$accheck rate
#learn.fit_one_cycle(6, lr_max=1e-5)

#### Discriminative Learning Rates

In [ ]:
#$accheck rate
'''learn = cnn_learner(dls, resnet18, metrics=error_rate)
learn.fit_one_cycle(3, 5e-3)
learn.unfreeze()
learn.fit_one_cycle(12, lr_max=slice(1e-6,1e-4))'''

In [ ]:
learn.recorder.plot_loss()

### 2.2 Using fast.ai cnn learner, with pretrained model Resnet18

In [ ]:
learn1 = cnn_learner(dls, resnet18, metrics=error_rate)
learn1.fine_tune(2)

#### 2.2a Check Model Performance
on the validation set

In [ ]:
#preview predictions
preds1,_ = learn1.get_preds(dl=[(x,y)])
preds1[0]

In [ ]:
len(preds1[0]),preds1[0].sum()

In [ ]:
print("Metric: error rate")
learn1.metrics[0].value

In [ ]:
interp1 = ClassificationInterpretation.from_learner(learn1)
interp1.plot_confusion_matrix(figsize=(12,12), dpi=60)

In [ ]:
interp1.most_confused(min_val=6)

## 3. Evaluate Models on Testset
saved separately and not used for training and validation

In [ ]:
print("Metric: error rate")
learn1.metrics[0].value